In [1]:
import os
os.chdir("../")

import semiolog as slg

In [2]:
semiotic = slg.Cenematic("en_bnc")

SLG [I]: Checking config correctness... Config correct!


100%|██████████| 3/3 [00:00<00:00, 297.77it/s]

SLG [I]: Dataset loaded from disk (TXT files)
SLG [I]: Vocabulary loaded from disk


  0%|          | 0/25 [00:00<?, ?it/s]

SLG [I]: nGrams loaded from disk (['2_5.json', '3_5.json', '4_5.json', '5_5.json', '6_5.json', '7_5.json', '8_5.json', '9_5.json', '10_5.json', '11_5.json', '12_5.json', '13_5.json', '14_5.json', '15_5.json', '16_5.json', '17_5.json', '18_5.json', '19_5.json', '20_5.json', '21_5.json', '22_5.json', '23_5.json', '24_5.json', '25_5.json', '26_5.json'])


2022-05-01 13:27:41.610236: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-05-01 13:27:41.610464: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



All model checkpoint layers were used when initializing TFBertForMaskedLM.

All the layers of TFBertForMaskedLM were initialized from the model checkpoint at models/en_bnc/paradigms/tf_model.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


SLG [I]: Paradigmatizer loaded from disk


In [3]:
from collections import defaultdict, Counter
import networkx as nx
from ipywidgets import interact
from math import log, log2, log1p, exp
import numpy as np
from tqdm.notebook import tqdm, trange
from scipy.stats import entropy

In [4]:
norm_factor = 1/sum(semiotic.vocab.ng1.freq.values())

In [5]:
ng_atts = [att for att in dir(semiotic.vocab) if att.startswith("ng")]

In [30]:
ng_freq = {i:eval(f"semiotic.vocab.{att}.freq") for i,att in enumerate(ng_atts)}

In [82]:
ng_prob = [{k:v*norm_factor for k,v in eval(f"semiotic.vocab.{att}.freq.items()")} for att in ng_atts]
ng_prob = {len(next(iter(ng_i.keys()))):ng_i for ng_i in ng_prob}

In [98]:
200 in ng_prob.keys()

False

In [87]:
def build_graph_data(string,ng_prob):

    edge_data = []
    for beginning in range(0, len(string)):
        for end in range(beginning + 1, len(string) + 1):
            subsequence_label = string[beginning:end]
            if len(subsequence_label)>len(ng_prob) or subsequence_label not in ng_prob[len(subsequence_label)] or subsequence_label == string:
                print(subsequence_label)
                continue
            edge_data.append(
                (
                    beginning,
                    end,
                    {
                        "label": subsequence_label,
                    },
                )
            )

    return edge_data

In [91]:
from semiolog.util import subsequences
from math import log2
from itertools import islice

def ent_morph(pre_morph):
    ind_l = {k[0]:v for k,v in ng_freq[len(pre_morph)].items() if k[1:]==pre_morph}
    ind_r = {k[-1]:v for k,v in ng_freq[len(pre_morph)].items() if k[:-1]==pre_morph}
    return entropy(list(ind_l.values()),base=len(ng_freq[0])) * entropy(list(ind_r.values()),base=len(ng_freq[0]))

def chain2seq(string, ng_prob):
    
    lSt = len(string)

    # If sent of len <2r, then return the interval
    if lSt<2:
        return [(0,lSt)]
    elif lSt == 2:
        return [(0,1),(1,2)]

    # If a character in the string not in vocab, add it
    for c in string:
        if c not in ng_prob[1]:
            ng_prob[1][c]=0
            # self.voc_rank[c]=(len(self.voc)+1)**self.zipf_factor

    graph_data = build_graph_data(string, ng_prob)
    seg_graph_full = nx.DiGraph()
    seg_graph_full.add_edges_from(graph_data)

    # Construct weights

    if lSt > len(ng_prob):
        normalizer = 0
    else:
        normalizer = ng_prob[lSt].get(string,0)

    for edge in seg_graph_full.edges:
        label = seg_graph_full.edges[edge]["label"]
        score = ng_prob[len(label)].get(label,0)-normalizer
        # score = ent_morph(label)
        
        if score == 0:#.000005:
            score = float("inf")
        else:
            score = -log(score)
        print((label,score))

        seg_graph_full.edges[edge]["weight"] = score

    # Find best segmentation out of shortest path


    # def k_shortest_paths(G, source, target, k, weight=None):
    #     k_s = list(islice(nx.shortest_simple_paths(G, source, target, weight=weight), k))
    #     k_l = [[string[l:r] for l,r in subsequences(sp, 2)] for sp in k_s]
    #     k_sc = [[log(ng_prob[len(label)-1][label]) for label in k_] for k_ in k_l]
    #     return (k_s,k_l,k_sc)

    # k_short = k_shortest_paths(seg_graph_full, 0, lSt, 5)

    # shortest_path = nx.shortest_path(seg_graph_full, 0, lSt, weight="weight", method="dijkstra")
    shortest_path = nx.shortest_path(seg_graph_full, 0, lSt, weight="weight", method="bellman-ford")
    # shortest_path_length = nx.shortest_path_length(seg_graph_full, 0, lSt, weight="weight",method="bellman-ford")

    seg_offsets = subsequences(shortest_path, 2)

    print(seg_offsets)
    # splits = []
    # for start,end in seg_offsets:
    #     splits.append(string[start:end])

    return seg_offsets

In [92]:
from random import randint
sent = semiotic.corpus.train[randint(0,100000)]["text"]
# sent = "I have made my plans and I must stick to them"
sent = "thisisverygood"
# sent = semiotic_old.corpus.train[randint(0,100000)]["text"]
print(sent)
sent = semiotic.syntagmatic.tokenizer.normalizer.normalize_str(sent)

thisisverygood


In [93]:
import graphviz as gv

# @interact(norm_factor_param=(0,1,0.01))
# def show_norm_factor(norm_factor_param = norm_factor):

G = nx.DiGraph()

to_segment = [(sent,(0,len(sent)))]

while len(to_segment)>0:
    for segment, offset in to_segment:

        seg_offsets = chain2seq(segment,ng_prob)

        for offset_l,offset_r in seg_offsets:
            G.add_edge((segment,offset), (segment[offset_l:offset_r],(offset[0]+offset_l,offset[0]+offset_r)))
        
        to_segment = [x for x in G.nodes() if G.out_degree(x)==0 and G.in_degree(x)==1]
        to_segment = [seg for seg in to_segment if len(seg[0])>2]

def plot_graph(seg_tree):
    # seg_tree = G
    seg_tree_graph = gv.Digraph(name="seg_tree")

    tree_nodes_list = [(str(i), l) for l, i in list(seg_tree.nodes)]
    for node in tree_nodes_list:
        seg_tree_graph.node(
            *node, color="white", fontsize="20", fontname="RobotoMono-Thin"
        )  # style="filled", color="grey")
    seg_tree_graph.attr("edge", color="slategrey")
    seg_tree_graph.edges([(str(p[1]), str(c[1])) for p, c in list(seg_tree.edges)])
    return seg_tree_graph

# seg_tree_graph
@interact
def inter():
    # return seg_tree_graph
    return plot_graph(G)

thisisverygood
('t', 2.392038146663398)
('th', 3.5673587347573292)
('thi', 6.095057997710096)
('this', 6.805767355077965)
('thisi', 8.815797410878355)
('thisis', 9.08560664855282)
('thisisv', 13.834400874623787)
('thisisve', 14.038968040365061)
('thisisver', 14.042040239402032)
('thisisvery', 14.06696364785449)
('thisisveryg', 18.72725313306366)
('thisisverygo', inf)
('thisisverygoo', inf)
('h', 2.9589219300469978)
('hi', 5.063567517108293)
('his', 6.146982473299833)
('hisi', 8.67645250967408)
('hisis', 9.080465915370585)
('hisisv', 13.834400874623787)
('hisisve', 14.038968040365061)
('hisisver', 14.042040239402032)
('hisisvery', 14.06696364785449)
('hisisveryg', 18.72725313306366)
('hisisverygo', inf)
('hisisverygoo', inf)
('hisisverygood', inf)
('i', 2.6270179346445004)
('is', 4.770955361393099)
('isi', 7.272009748434516)
('isis', 8.877219101261622)
('isisv', 13.671007327715351)
('isisve', 13.877830432551123)
('isisver', 13.880444813125195)
('isisvery', 13.909663359124334)
('isisvery

interactive(children=(Output(),), _dom_classes=('widget-interact',))

In [96]:
leaves = sorted([x for x in G.nodes() if G.out_degree(x)==0 and G.in_degree(x)==1],key=lambda x:x[1])
[k for k,o in leaves]

['m',
 'r',
 'g',
 'wh',
 'i',
 't',
 'e',
 'an',
 'd',
 'm',
 'is',
 's',
 'a',
 'in',
 'g',
 'ha',
 'm',
 's',
 'mi',
 'th']

In [31]:
from networkx.algorithms.traversal.depth_first_search import dfs_tree

node_3 = [node for node in G.nodes() if len(node[0])==3]

@interact(n=(0,len(node_3)-1))
def inter(n=0):
    # return seg_tree_graph

    subtree = dfs_tree(G, node_3[n])
    return plot_graph(subtree)

interactive(children=(IntSlider(value=0, description='n', max=7), Output()), _dom_classes=('widget-interact',)…

In [29]:
node_3

[('bus', (9, 12)),
 ('con', (67, 70)),
 ('par', (50, 53)),
 ('res', (1, 4)),
 ('res', (71, 74)),
 ('ide', (4, 7)),
 ('emo', (41, 44)),
 ('ubl', (16, 19))]

In [79]:
sent_seq = semiotic(sent)
sent_seq.tree.plot()

interactive(children=(Output(),), _dom_classes=('widget-interact',))

<function semiolog.syntagmatic.tree.Tree.plot.<locals>.inter()>

In [60]:
sent = "ked"

seg_candidates = []
segs = [[sent[:n],sent[n:]]for n in range(1,len(sent))]
for segl,segr in segs:
    prob_ab = ng_prob[len(sent)-1].get(sent,0)
    prob_a = ng_prob[len(segl)-1].get(segl,0)-prob_ab
    prob_b = ng_prob[len(segr)-1].get(segr,0)-prob_ab
    
    if prob_a*prob_b*prob_ab == 0:
        seg_candidates.append(((segl,segr),float("inf")))
    else:
        seg_candidates.append(((segl,segr),log(prob_ab/(prob_a*prob_b))))
sorted(seg_candidates,key=lambda x:x[-1])

[(('ke', 'd'), 1.721919945722911), (('k', 'ed'), 1.786102436173875)]

In [33]:
norm_factor = 1/500

af = 40
abf = 10
a = af*norm_factor
ab = abf*norm_factor

In [34]:
a

0.08

In [35]:
a-ab

0.06

In [32]:
20*norm_factor

0.04

In [8]:
ng_prob_all = defaultdict()
for dic in ng_prob:
    ng_prob_all.update(dic)

In [9]:
top_dict = sorted(ng_prob_all.items(),key=lambda x:x[-1],reverse=True)

In [16]:
top_dict_30k = dict(top_dict[:30000])

In [36]:
list(top_dict_30k.items())[:20]

[('e', 0.12283516959154912),
 ('t', 0.09144317330136345),
 ('a', 0.07986945016799842),
 ('o', 0.07554999657567639),
 ('i', 0.07229377970004593),
 ('n', 0.07001552324870404),
 ('s', 0.06436566038490976),
 ('r', 0.0612647297217823),
 ('h', 0.05187486568481037),
 ('l', 0.04127051180816118),
 ('d', 0.038279210527240905),
 ('c', 0.031054477724511854),
 ('th', 0.028230373026901415),
 ('u', 0.027912862902896105),
 ('m', 0.02468633977149756),
 ('he', 0.02425154868000255),
 ('f', 0.022145402606558767),
 ('p', 0.020483912012048574),
 ('g', 0.019864474601394295),
 ('w', 0.01915778851149281)]

In [43]:
semiotic.vocab.freq

{'[PAD]': 0,
 '[UNK]': 0,
 '[CLS]': 0,
 '[SEP]': 0,
 '[MASK]': 0,
 'and': 449402,
 'the': 327925,
 'to': 270003,
 'in': 268528,
 'of': 262663,
 'is': 250238,
 'for': 233607,
 'was': 233402,
 'that': 228044,
 's': 221072,
 'as': 220829,
 'ofthe': 208676,
 'inthe': 206767,
 'on': 195018,
 'ing': 189035,
 'with': 187535,
 'at': 180648,
 'or': 178559,
 'ed': 176854,
 'by': 171965,
 'it': 168664,
 'sand': 163379,
 'but': 162439,
 'an': 156652,
 'this': 152781,
 'from': 152185,
 'are': 147556,
 'er': 145331,
 'which': 126484,
 'its': 123544,
 'es': 112030,
 'all': 110182,
 'you': 108126,
 'tothe': 102708,
 'then': 102338,
 'a': 100482,
 'his': 100149,
 'i': 99805,
 'y': 99670,
 'sof': 97206,
 'had': 96299,
 'al': 96283,
 'un': 95597,
 'her': 93987,
 'them': 91173,
 'they': 91167,
 'if': 90311,
 'their': 89843,
 'onthe': 89784,
 'when': 88966,
 'there': 88337,
 'one': 83417,
 'ers': 82705,
 'm': 79998,
 'not': 79982,
 'andthe': 79907,
 'itwas': 79579,
 'ad': 79275,
 'en': 78028,
 'us': 77216,

In [87]:
def ent_morph(pre_morph):
    ind_l = {k[0]:v for k,v in ng_freq[len(pre_morph)].items() if k[1:]==pre_morph}
    ind_r = {k[-1]:v for k,v in ng_freq[len(pre_morph)].items() if k[:-1]==pre_morph}
    return (entropy(list(ind_l.values()),base=base),entropy(list(ind_r.values()),base=len(ng_freq[0])))

In [91]:
ent_morph("ntropys")

(0.0, 0.0)

In [58]:
pre_morph = "neofu"
base = len(ng_freq[0])

ng_ent = []
for n in trange(5):
    ent = defaultdict()
    for pre_morph in tqdm(ng_freq[n].keys()):
        ind_l = {k[0]:v for k,v in ng_freq[len(pre_morph)].items() if k[1:]==pre_morph}
        ind_r = {k[-1]:v for k,v in ng_freq[len(pre_morph)].items() if k[:-1]==pre_morph}
        ent[pre_morph] = (entropy(list(ind_l.values()),base=base),entropy(list(ind_r.values()),base=base))
    ng_ent.append(ent)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/158 [00:00<?, ?it/s]

  0%|          | 0/2802 [00:00<?, ?it/s]

  0%|          | 0/37740 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [86]:
sorted([(-v1*v2,k) for k,(v1,v2) in ent.items() if {str(n) for n in range(10)}.intersection(set(k))==set()])

[(-0.32153593067087166, 'iis'),
 (-0.3146323019459895, 'its'),
 (-0.3144086754239838, 'one'),
 (-0.3136972877297587, 'iii'),
 (-0.30924218640845647, 'and'),
 (-0.3059351020406778, 'ane'),
 (-0.30456975496373817, 'eto'),
 (-0.3035706023475298, 'the'),
 (-0.2968392298505186, 'ley'),
 (-0.29657875679441625, 'mrs'),
 (-0.29519421401388846, 'ing'),
 (-0.29232829690451295, 'are'),
 (-0.29102537033886217, 'plc'),
 (-0.2880426027308801, 'sby'),
 (-0.2872174124603174, 'ale'),
 (-0.28491350679665556, 'acs'),
 (-0.28485137136086736, 'eie'),
 (-0.28442219107375794, 'sto'),
 (-0.2834905205668527, 'iin'),
 (-0.283315868026601, 'dry'),
 (-0.2832646198251599, 'ton'),
 (-0.28064128140704386, 'aen'),
 (-0.2804786797472023, 'eby'),
 (-0.2787753734417804, 'bay'),
 (-0.27873644014722043, 'ine'),
 (-0.27820120325089437, 'joe'),
 (-0.2778155528035592, 'has'),
 (-0.2777190150666532, 'you'),
 (-0.2768620211645106, 'god'),
 (-0.2763094353690957, 'ise'),
 (-0.2759258887400977, 'age'),
 (-0.2752560496804605, 'bad

In [71]:
{str(n) for n in range(10)}.intersection(set("bla"))

set()

In [61]:
ng_ent[1]

defaultdict(None,
            {'th': (0.5418297737573956, 0.2592122575696496),
             'he': (0.20657174239011736, 0.5754732328416872),
             'in': (0.5704666467499032, 0.4646195365273106),
             'er': (0.4941592895371537, 0.5358303571920235),
             'an': (0.5708498878765635, 0.4236488366259263),
             're': (0.5033318681479382, 0.5607446705467417),
             'on': (0.46331221377647896, 0.5297807676511183),
             'es': (0.5369668621995007, 0.5142152139843394),
             'nt': (0.3182424037599364, 0.487683515293638),
             'en': (0.5428593582854683, 0.462823121841742),
             'st': (0.4928195578394452, 0.4887170023516663),
             'at': (0.5104371240282828, 0.47269422948978),
             'ti': (0.4532752184268039, 0.4359193410138439),
             'ed': (0.5473678783128753, 0.5348685967273655),
             'nd': (0.22059135785589637, 0.5605612254627312),
             'to': (0.5287761002122243, 0.576319426939956),
        

In [47]:
from scipy.stats import entropy
import numpy as np